In [ ]:
!pip install torch_geometric
!pip install captum

from google.colab import files
files.upload()

In [ ]:
#jupytext --to ipynb Comparative_Cheb_GCN.py
# Don't forget to upload model as well
!unzip numpy_data.zip

In [ ]:
import time
start_time = time.time()


!pip install torch_geometric
!pip install pdf2image
!apt-get install poppler-utils
# !pip install captum

from google.colab import files
files.upload()

jupytext --to ipynb Comparative_Cheb_GCN.py
Don't forget to upload model.pth as well
!unzip numpy_data.zip

In [ ]:
from torch_geometric.data import Data, DataLoader
from torch_geometric.datasets import TUDataset, Planetoid
from torch_geometric.nn import GCNConv, Set2Set
from torch_geometric.explain import GNNExplainer
import torch_geometric.transforms as T
import torch
#import torch.nn.functional as F
import os
from tqdm import tqdm, trange
#
import matplotlib.pyplot as plt

In [ ]:
#Loading saved model architecture
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import ChebConv
from torch_geometric.nn import GATConv
from torch_geometric.nn import TransformerConv
from torch_geometric.nn import global_mean_pool
from torch_geometric.nn.pool import global_max_pool

In [ ]:
class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        # self.conv1 = GCNConv(6, 512, improved = True)
        # self.conv2 = GCNConv(512, 256, improved = True)

        # self.conv1 = ChebConv(6, 512, K = 2)
        # self.conv2 = ChebConv(512,256, K = 2)

        self.conv1 = ChebConv(6, 256, K = 3)
        self.conv2 = ChebConv(256,256, K = 4)
        self.conv3 = ChebConv(256, 256, K = 5)
        # self.fc1 = Linear(256, 512)
        # self.fc2 = Linear(512, 256)

        # self.conv1 = GATConv(6, 256, heads = 4)
        # self.conv2 = GATConv(256*4,64, heads = 1, concat=False)

        # self.conv1 = TransformerConv(6, 200, heads = 3)
        # self.conv2 = TransformerConv(200*3,256, heads = 1, concat=False)

        self.lin = Linear(256, 120)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings
        x = self.conv1(x, edge_index)
        x = x.relu()

        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)
        x = x.relu()

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
        # = global_max_pool(x, batch)  # [batch_size, hidden_channels]

        # x = self.fc1(x)
        # x = x.relu()
        # x = self.fc2(x)
        # x = x.relu()

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.2, training=self.training) #0.5
        x = self.lin(x)

        return x

In [ ]:
model = GCN(hidden_channels=464)
print(model)

In [ ]:
# print(os.getcwd()+ "/model.pth")
# model_save_location = os.getcwd()+ "/model.pth"
print(os.getcwd()+ "/model_123.pt")
model_save_location = os.getcwd()+ "/model_123.pt"
model.load_state_dict(torch.load(model_save_location, map_location=torch.device('cpu')))
print(model.conv3)
print(model.state_dict())

In [ ]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import os
import torch
import networkx as nx
os.environ['TORCH'] = torch.__version__
print(torch.__version__)
print(os.getcwd())

In [ ]:
# +
import time
#Extract Training Data
#Loads IEEE34 Bus Simulation Data

In [ ]:
path = os.getcwd()

In [ ]:
numpy_data = np.load(path+"/new_data.npy",allow_pickle = True)
numpy_data = np.squeeze(numpy_data)
print(numpy_data)
print(numpy_data.shape)
print(numpy_data.size)

In [ ]:
value_data = np.load(path+"/new_labels.npy",allow_pickle = True)
print(value_data)
print(value_data.shape)
print(value_data.size)

In [ ]:
NodeIndex = np.load(path+"/NodeIndex.npy",allow_pickle = True)
print(NodeIndex)
print(NodeIndex.shape)
print(NodeIndex.size)

In [ ]:
AdjacencyMatrix = np.load(path+"/AdjacencyMatrix.npy",allow_pickle = True)
print(AdjacencyMatrix)
print(AdjacencyMatrix.shape)
print(AdjacencyMatrix.size)

In [ ]:
# +
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.transforms import NormalizeFeatures
from torch.nn.functional import normalize

In [ ]:
encoder = ['150', '150R', '149', '1', '2', '3', '7', '4', '5', '6', '8', '12',
'9', '13', '9R', '14', '34', '18', '11', '10', '15', '16', '17', '19',
'21', '20', '22', '23', '24', '25', '25R', '26', '28', '27', '31',
'33', '29', '30', '250', '32', '35', '36', '40', '37', '38', '39',
'41', '42', '43', '44', '45', '47', '46', '48', '49', '50', '51',
'151', '52', '53', '54', '55', '57', '56', '58', '60', '59', '61',
'62', '63', '64', '65', '66', '67', '68', '72', '97', '69', '70',
'71', '73', '76', '74', '75', '77', '86', '78', '79', '80', '81',
'82', '84', '83', '85', '87', '88', '89', '90', '91', '92', '93',
'94', '95', '96', '98', '99', '100', '450', '197', '101', '102',
'105', '103', '104', '106', '108', '107', '109', '300', '110',
'111', '112', '113', '114', '135', '152', '160R', '160', '61S', '610']
#encoder = ['SourceBus', '800', '802', '806', '808', '810', '812', '814', '814r', '850',
#'816', '818', '824', '820', '822', '826', '828', '830', '854', '832', '858',
#'834', '860', '842', '836', '840', '862', '844', '846', '848', '852r', '888', '856', '852', '864', '838', '890']
print(encoder)
print(len(encoder))
#research_paper_decoder = [0,0,1,2,3,4,5,6,6,6,6,7,8,9,10,11,8,12,12,13,14,15,21,15,16,16,16,17,18,18,13,13,19,13,20,22,23]
research_paper_decoder = [0,0,0,1,2,3,4,5,6,7,8,9,10,11,10,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,72,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,14,11,61,61,63,119]
print(research_paper_decoder)
print(len(research_paper_decoder))
FaultLocationLabels = value_data[:,3]

In [ ]:
for n in range(len(FaultLocationLabels)):
    FaultLocationLabels[n]=research_paper_decoder[encoder.index(str(FaultLocationLabels[n]).upper())]

In [ ]:
y = FaultLocationLabels.astype("int64")
y = torch.from_numpy(y)
x = numpy_data.astype("float32")
x = torch.from_numpy(x)

In [ ]:
NodeIndex = NodeIndex.astype("int64")
NodeIndex = NodeIndex.T
NodeIndex = torch.from_numpy(NodeIndex)

In [ ]:
print(y)
print(y.dtype)
print(x)
print(x.dtype)
print(NodeIndex)
print(NodeIndex.dtype)
#
print(x[0])
print(normalize(x[0]))
#
result_translator = np.unique(FaultLocationLabels.astype("int64")).tolist()
print()
total_data_list = []
for n in range(len(x)):
    #print(x[n])
    #print(y[n])
    DataObject = Data(x = x[n], edge_index = NodeIndex, y = y[n], is_undirected = True) #Testing with non-normalized data
    #DataObject = Data(x = x[n], edge_index = NodeIndex, y = y[n], is_undirected = True)
    DataObject.is_undirected = True
    total_data_list.append(DataObject)
#print('Y'*888)
#print(total_data_list[0].x)

In [ ]:
print()
#print(f'Dataset: {total_data_list}:')
print('===================')
print(f'Number of graphs: {len(total_data_list)}')
print(f'Number of features: {total_data_list[0].num_features}')
#print(f'Number of classes: {total_data_list[0].num_classes}')

In [ ]:
data = total_data_list[0]  # Get the first graph object.
#print(data)
#print(data.y)

In [ ]:
print()
print(NormalizeFeatures(data.x))
print(data.x)
print('=============================================================')

In [ ]:
# Gather some statistics about the first graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

In [ ]:
from random import shuffle
torch.manual_seed(12345)
#total_data_list = total_data_list.shuffle()
shuffle(total_data_list)

In [ ]:
train_dataset = total_data_list[:26000] #9150 is half
test_dataset = total_data_list[26000:]

In [ ]:
print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')

In [ ]:
print(test_dataset[0].x)
print(test_dataset[0])
#print(train_dataset[0].x)

In [ ]:
for sample in range(len(test_dataset)):
    # noise = np.random.normal(1,0.03, size = (37,6)) #0.09 #Uncomment for noise
    # noise = noise.astype("float32")
    # noise = torch.from_numpy(noise)
    # test_dataset[sample].x = noise*test_dataset[sample].x
    test_dataset[sample].x = normalize(test_dataset[sample].x)
    #print(sample)

In [ ]:
print('Y'*888)
#print(train_dataset[0].x)
print(test_dataset[0].x)
print(test_dataset[0])
##noise = np.random.normal(1,0.09, size = (16,6))
##noise = noise.astype("float32")
##noise = torch.from_numpy(noise)
print('=============================================================')
##
print(train_dataset[0].x)
print(train_dataset[0])
#print(train_dataset[0].x)

In [ ]:
for sample in range(len(train_dataset)):
#    noise = np.random.normal(1,0.09, size = (16,6))
#    #    noise = np.random.normal(1,0.09, size = (1,96))
#    noise = noise.astype("float32")
#    noise = torch.from_numpy(noise)
##    print(noise)
##    X_test[sample] = X_test[sample]*noise[0]
#    train_dataset[sample].x = noise*train_dataset[sample].x
    train_dataset[sample].x = normalize(train_dataset[sample].x)
    #print(sample)

In [ ]:
print('Y'*888)
#print(train_dataset[0].x)
print(train_dataset[0].x)
print(train_dataset[0])
##print(noise*test_dataset[0].x)
# Gather some statistics about the first graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
#AUC Metric for PyTorch Models
#from torcheval.metrics.aggregation.auc import AUC
#metric = AUC()
from sklearn.metrics import roc_auc_score
from torch.nn import Softmax

In [ ]:
Soft_list = np.array([[]]).reshape((0,120))
values_list = np.array([])
pred_list = np.array([])

In [ ]:
with torch.no_grad():
    model.eval()
    for data in test_loader:
        out = model(data.x, data.edge_index, data.batch)
        pred= out.argmax(dim=1)
        softmax = Softmax()
        softout = softmax(out)
        y = data.y
        Soft_list = np.concatenate((Soft_list, softout.numpy()), axis=0)
        pred_list = np.concatenate((pred_list, pred.numpy()), axis=0)
        # print(Soft_list)
        values_list= np.concatenate((values_list, y.numpy()), axis=None)
   # print(out)
   # print('*'*888)
   # print(softout)
   # print('*'*888)
print(Soft_list)
print(Soft_list.shape)
print(values_list)
print(values_list.shape)
print(pred_list)
print(pred_list.shape)

print("The Area Under ROC Curve (AUC) =", roc_auc_score(values_list, Soft_list, multi_class='ovr'))

from sklearn.metrics import f1_score
print("F1 Score =", f1_score(values_list, pred_list, average = 'macro'))

from sklearn.metrics import accuracy_score

print("Testing Accuracy =",accuracy_score(values_list, pred_list))

# Confusion Matrix for PyTorch Models
encoder = ['SourceBus', '800', '802', '806', '808', '810', '812', '814', '814r', '850',
'816', '818', '824', '820', '822', '826', '828', '830', '854', '832', '858',
'834', '860', '842', '836', '840', '862', '844', '846', '848', '852r', '888', '856', '852', '864', '838', '890']
print(encoder)

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

confusion_mat_labels = ["Src/800", "802","806","808","810","812","814/850/816","818","824/828","820","822","826","830/854","852/832/888","858",
       "834/842","836/840/862","844","846/848","856","864","860","838","890"]

values_list = values_list.astype(object)
pred_list = pred_list.astype(object)

for faults in range(24):
   pred_list[pred_list==faults] = confusion_mat_labels[faults]
   values_list[values_list==faults] = confusion_mat_labels[faults]

print(confusion_mat_labels)
print(np.unique(values_list))
print(np.unique(pred_list))


# cm = confusion_matrix(values_list, pred_list, labels=confusion_mat_labels)#, labels=classes
# ConfusionMatrixDisplay(cm, display_labels=confusion_mat_labels).plot(xticks_rotation = 35).ax_.set_title("Cheb GCN Confusion Matrix")
# plt.show()
cm = confusion_matrix(values_list, pred_list,  labels=confusion_mat_labels)#, labels=classes
cmp = ConfusionMatrixDisplay(cm, display_labels=confusion_mat_labels)#.plot(xticks_rotation = 35)#.ax_.set_title("kNN Confusion Matrix")
fig, ax = plt.subplots(figsize=(12, 12))
#ax.set_xticks(xtick_labels-12)
cmp.plot(ax=ax, xticks_rotation = 40)
plt.xticks(np.arange(len(confusion_mat_labels)), confusion_mat_labels, ha='right')
plt.show()


# -

print(model.state_dict())
torch.save(model.state_dict(), 'model_2.pt')
# torch.save(model, 'entire_model.pt')

cm = confusion_matrix(values_list, pred_list,  labels=confusion_mat_labels)#, labels=classes
cmp = ConfusionMatrixDisplay(cm, display_labels=confusion_mat_labels)#.plot(xticks_rotation = 35)#.ax_.set_title("kNN Confusion Matrix")
fig, ax = plt.subplots(figsize=(12, 12))
#ax.set_xticks(xtick_labels-12)
cmp.plot(ax=ax, xticks_rotation = 40)
plt.xticks(np.arange(len(confusion_mat_labels)), confusion_mat_labels, ha='right')
plt.show()